## Resources
tutorial: https://www.youtube.com/watch?v=kCc8FmEb1nY

In [1]:
import torch
import torch.nn.functional as F
import torch.optim as optim
import torch.nn as nn
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from tokenizer import SimpleTokenizer
torch.manual_seed(1337)
from dataset import SpeechesClassificationDataset
from tokenizer import SimpleTokenizer
from transformer import CompositeEmbedding, CustomTransformerEncoder

import nltk
# nltk.download('punkt', download_dir="../data/nltk_punkt")
# nltk.data.path.append("../data/nltk_punkt")

## Scratch Space

In [2]:
t1 = torch.rand((1, 3, 3))
t2 = torch.rand(size=(1, 4, 3))
torch.bmm(t1, t2.transpose(1, 2))

tensor([[[0.7335, 0.9315, 0.7578, 0.7938],
         [0.2683, 0.2031, 0.2434, 0.3224],
         [0.7478, 0.7225, 0.6855, 0.7379]]])

In [3]:
x = torch.tensor([1, 2, 3])
torch.arange(x.size(-1)).expand_as(x)

tensor([0, 1, 2])

In [4]:
with open("../data/speechesdataset/train_LM.txt", "r") as f:
    text = f.read()


# scd = SpeechesClassificationDataset(tokenizer=tok, file_path="data/train_CLS.tsv")

In [5]:
tok = SimpleTokenizer(text=text)

In [6]:
# Sample input: batch size of 3, sequence length of 5

vocab_size = tok.vocab_size  # Example vocabulary size
n_embed = 64    # Example embedding dimension
embedding = CompositeEmbedding(vocab_size, n_embed)

sample_indices = torch.randint(0, vocab_size, (3, 5)).to(device)


In [7]:
sample_indices

tensor([[2864, 4029, 4024, 2205, 4007],
        [1656, 1845, 2248, 2023, 3120],
        [ 621, 3543, 2456, 2936, 3478]], device='cuda:0')

In [8]:
e= embedding.to(device)

In [9]:
r = e(sample_indices)

In [10]:
r.shape

torch.Size([3, 5, 64])

In [11]:
vocab_size = tok.vocab_size
n_embed = 64       
n_heads = 2       
n_layer = 4       
n_input = 64      
n_hidden = 100  
n_output = 3 
batch_size = 16
seq_length = 10

In [12]:
model = CustomTransformerEncoder(vocab_size, n_embed, n_heads, n_layer, n_input, n_hidden, n_output).to(device)
model.eval()

CustomTransformerEncoder(
  (embedding): CompositeEmbedding(
    (token_embeddings): Embedding(4526, 64)
    (positional_embeddings): Embedding(4526, 64)
    (normalize): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
  )
  (transformer_layers): ModuleList(
    (0-3): 4 x NHeadAttention(
      (query): Linear(in_features=64, out_features=64, bias=True)
      (key): Linear(in_features=64, out_features=64, bias=True)
      (value): Linear(in_features=64, out_features=64, bias=True)
      (normalize): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.5, inplace=False)
    )
  )
  (ffnet): Sequential(
    (0): Linear(in_features=64, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=3, bias=True)
  )
)

In [13]:
input_tensor = torch.randint(low=0, high=vocab_size, size=(batch_size, seq_length), dtype=torch.long).to(device)

In [14]:
input_tensor

tensor([[ 114, 4241, 2757, 2751, 2242, 2215, 2749, 2657, 3674, 3088],
        [2347, 4407, 1953, 4194, 1541, 4301,  200, 2172, 1281, 1181],
        [2027, 2721, 4017, 3374, 2137,  357, 3910, 3827,  247, 1339],
        [1522, 1857,  934, 1695, 3293,  919,  892, 3648, 4262, 3405],
        [1525,  639, 2941, 3802, 3506,  380, 1274, 1817, 3798, 4470],
        [ 911, 1958, 3822, 1788, 4084, 2874,  437, 3616, 4366, 3618],
        [ 144, 2560, 3064, 2810, 3198,  387, 2399,  999, 1077,  338],
        [ 106, 1333,  955, 2622, 2010, 2753, 3636, 1735, 2454, 3344],
        [2305, 2334, 2190, 3269, 4040, 1261,  215, 2282, 2630, 1529],
        [2582, 3653, 1921, 2019, 3587, 3724, 4105, 3925, 2662, 2865],
        [ 319,  868, 3815, 2109, 2796, 3295, 4194, 1052, 1158, 1300],
        [2973, 3971,  502, 4099, 1038, 4457, 2095,  716, 1967, 2196],
        [2040, 4346, 4360, 4156, 2789, 1783, 4480, 1426, 3215, 1591],
        [4426, 2265, 2589, 3506, 2359, 2071, 2067,  957, 2141, 1622],
        [1983, 1778,

In [15]:
res = model(input_tensor)

In [16]:
res

tensor([[ 0.0009, -0.5531,  0.2170],
        [-0.0300, -0.2346,  0.0526],
        [-0.0867, -0.3334, -0.0536],
        [ 0.0575, -0.2837,  0.2274],
        [ 0.1406, -0.5387,  0.1170],
        [ 0.0458, -0.3529,  0.2439],
        [-0.0666, -0.3641,  0.2693],
        [ 0.0775, -0.3704,  0.0687],
        [ 0.1858, -0.4392,  0.2118],
        [ 0.0761, -0.1543,  0.1010],
        [ 0.0364, -0.5550, -0.0031],
        [ 0.2411, -0.0917, -0.1485],
        [-0.0159, -0.1915,  0.1488],
        [ 0.0188, -0.3322,  0.3497],
        [ 0.1658, -0.3319, -0.0485],
        [-0.1397, -0.0781,  0.1208]], device='cuda:0',
       grad_fn=<AddmmBackward0>)

## Model Training

In [17]:

seed = 42
""" Hyperparameters to use for training to roughly match 
the numbers mentioned in the assignment description """
batch_size = 16  # Number of independent sequences  we will process in parallel
block_size = 32  # Maximum context length for predictions
learning_rate = 1e-3  # Learning rate for the optimizer
n_embd = 64  # Embedding dimension
n_head = 2  # Number of attention heads
n_layer = 4  # Number of transformer layers


eval_interval = 100  # How often to evaluate train and test perplexity during training
max_iters = 500 # For language modeling, we can process all the batches for the entire dataset, but that takes a while, so we'll limit it to 500 iterations. For batch size of 16 and block size of  32, this is roughly, this is  500 * 16 * 32 = 256000 tokens, SOTA LMs are trained on trillions of tokens, so this is a very small dataset.
eval_iters = 200  # Number of iterations to evaluate perplexity on the test set


## classifier training hyperparameters. It is a simple 1 hidden layer feedforward network, with input 
## size of 64, hidden size of 50 and output size of 3.

n_input = 64  # Input size for the classifier, should match the embedding size of the transformer
n_hidden = 100  # Hidden size for the classifier
n_output = 3  # Output size for the classifier, we have 3 classes
epochs_CLS = 15 # epochs for classifier training

In [18]:
#LOAD DATA
import os
from main import load_texts, DataLoader, collate_batch, LanguageModelingDataset
data_dir = os.path.join("..", "data", "speechesdataset")
print("Loading data and creating tokenizer ...")
texts = load_texts('../data/speechesdataset')
tokenizer = SimpleTokenizer(' '.join(texts)) # create a tokenizer from the data
print("Vocabulary size is", tokenizer.vocab_size)

train_CLS_dataset = SpeechesClassificationDataset(tokenizer, os.path.join(data_dir, "train_CLS.tsv"))
train_CLS_loader = DataLoader(train_CLS_dataset, batch_size=batch_size,collate_fn=collate_batch,shuffle=True)

Loading data and creating tokenizer ...
Vocabulary size is 5755


In [19]:
te = CustomTransformerEncoder(vocab_size, n_embed, n_heads, n_layer, n_input, n_hidden, n_output).to(device)
criterion = nn.CrossEntropyLoss()  # Suitable for classification tasks
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Learning rate might need tuning

In [20]:
inputfile = os.path.join(data_dir, "train_LM.txt")
with open(inputfile, 'r', encoding='utf-8') as f:
    lmtrainText = f.read()
train_LM_dataset = LanguageModelingDataset(tokenizer, lmtrainText,  block_size)
train_LM_loader = DataLoader(train_LM_dataset, batch_size=batch_size, shuffle=True)
    # for the classification  task, you will train for a fixed number of epochs like this:
for epoch in range(epochs_CLS):
    te.train()  # Set the model to training mode
    total_loss = 0
    for xb, yb in train_CLS_loader:
        xb, yb = xb.to(device), yb.to(device)  # Move data to the appropriate device
        print(f"Batch shapes - xb: {xb.shape}, yb: {yb.shape}")
        #zero out grads
        optimizer.zero_grad()
        print(f"zeroed out gradient")
        #model prediction  
        outputs = te(xb)  
        print(f"made prediction")
        #compute loss
        loss = criterion(outputs, yb)

        #backprop 
        loss.backward()
        #grad update  
        optimizer.step()  

        #update loss for this epoch
        total_loss += loss.item()  

    #print avg loss over epoch
    epoch_avg_loss = total_loss / len(train_CLS_loader)

    print(f"Epoch [{epoch+1}/{epochs_CLS}], Loss: {epoch_avg_loss:.4f}")

Batch shapes - xb: torch.Size([16, 32]), yb: torch.Size([16])


../aten/src/ATen/native/cuda/Indexing.cu:1289: indexSelectLargeIndex: block: [183,0,0], thread: [0,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1289: indexSelectLargeIndex: block: [183,0,0], thread: [1,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1289: indexSelectLargeIndex: block: [183,0,0], thread: [2,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1289: indexSelectLargeIndex: block: [183,0,0], thread: [3,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1289: indexSelectLargeIndex: block: [183,0,0], thread: [4,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1289: indexSelectLargeIndex: block: [183,0,0], thread: [5,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1289: indexSelectLargeIndex: block: [183,0,0], 

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
